# Sign Optimization for FCPM Director Reconstruction
## A Comprehensive Comparison of Advanced Methods

---

### The Problem

In nematic liquid crystals, the director **n** has a fundamental symmetry: **n ≡ -n**. After Q-tensor eigendecomposition, each voxel has a director with an **arbitrary sign**. We need to find consistent signs that produce a smooth, physically realistic field.

### Mathematical Formulation

**Objective**: Minimize gradient energy
$$E(S) = \sum_{(i,j) \in \text{neighbors}} |s_i \cdot \hat{n}_i - s_j \cdot \hat{n}_j|^2$$

where $s_i \in \{+1, -1\}$ for each voxel.

**Equivalence to Ising Model**:
$$E(S) = \sum_{(i,j)} J_{ij} \cdot s_i \cdot s_j + \text{const}$$

where $J_{ij} = -2(\hat{n}_i \cdot \hat{n}_j)$

---

### Methods Compared

| # | Method | Approach | Complexity |
|---|--------|----------|------------|
| 1 | V2 Layer+Refine | Greedy layer propagation | O(N) |
| 2 | Graph Cuts | Min-cut/max-flow (exact) | O(N log N) |
| 3 | Hierarchical | Coarse-to-fine pyramid | O(N) |
| 4 | Simulated Annealing | Stochastic optimization | O(iter × N) |
| 5 | Belief Propagation | Message passing | O(iter × N) |

In [1]:
# Setup and Imports
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import time

# Configure matplotlib for nice plots
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 11

# Add paths
sys.path.insert(0, str(Path('.').resolve().parent))
sys.path.insert(0, str(Path('.').resolve()))

import fcpm
from fcpm.core.director import DirectorField
from fcpm.reconstruction import reconstruct_via_qtensor

# Import all approaches
from sign_optimization_v2 import layer_then_refine, compute_gradient_energy
from approaches.graph_cuts import GraphCutsOptimizer
from approaches.simulated_annealing import SimulatedAnnealingOptimizer, SimulatedAnnealingConfig
from approaches.hierarchical import HierarchicalOptimizer
from approaches.belief_propagation import BeliefPropagationOptimizer, BeliefPropagationConfig

# Check PyMaxflow
gc = GraphCutsOptimizer()
print(f"PyMaxflow available: {gc._has_maxflow}")
print("All imports successful!")

PyMaxflow available: True
All imports successful!


---

## Part 1: Algorithm Descriptions

### 1.1 V2 Layer-by-Layer + Refinement

**Idea**: Propagate signs layer-by-layer along z-axis, then refine globally.

```
Phase 1: Layer Propagation
──────────────────────────
for z = 1 to nz-1:
    for each voxel (y,x) in layer z:
        if dot(n[y,x,z], n[y,x,z-1]) < 0:
            flip n[y,x,z]

Phase 2: Iterative Refinement  
─────────────────────────────
repeat until convergence:
    for each voxel:
        if flipping reduces energy:
            flip
```

**Pros**: Fast, simple | **Cons**: Gets stuck in local minima

### 1.2 Graph Cuts (Min-Cut/Max-Flow)

**Idea**: Model as binary labeling problem, solve exactly via max-flow.

```
Step 1: Build Graph
───────────────────
• Source S = "positive sign"
• Sink T = "negative sign"  
• Edge weights from director alignment

Step 2: Find Min-Cut
────────────────────
• Run Boykov-Kolmogorov algorithm
• Cut partitions voxels into S-side and T-side

Step 3: Assign Signs
────────────────────
• S-reachable → positive
• T-reachable → negative
```

**Pros**: Globally optimal, fast | **Cons**: Requires library

### 1.3 Hierarchical Coarse-to-Fine

**Idea**: Solve at coarse scale first, refine progressively.

```
Step 1: Build Pyramid
─────────────────────
Level 3: 32×32×16 (original)
Level 2: 16×16×8
Level 1: 8×8×4  
Level 0: 4×4×2 (coarsest)

Step 2: Coarsening (2×2×2 blocks)
─────────────────────────────────
• Average Q-tensors in block
• Find dominant eigenvector

Step 3: Bottom-Up Refinement
────────────────────────────
• Solve coarsest level
• Upsample signs to finer level
• Local refinement
• Repeat until original resolution
```

**Pros**: Fast, captures global structure | **Cons**: May smooth defects

### 1.4 Simulated Annealing

**Idea**: Stochastic search with temperature-controlled acceptance.

```
Initialize: T = T_high

repeat:
    pick random voxel
    compute ΔE if flipped
    
    if ΔE < 0:
        accept flip
    else:
        accept with probability exp(-ΔE/T)
    
    T ← α × T  (cooling)
    
until T < T_min
```

**Pros**: Can escape local minima | **Cons**: Very slow

### 1.5 Belief Propagation

**Idea**: Message passing on factor graph for probabilistic inference.

```
Initialize: all messages = 0.5

repeat:
    for each edge (u → v):
        m_{u→v}(s) = Σ ψ(s_u, s_v) × Π m_{w→u}(s_u)
    
    apply damping: m ← α×m_old + (1-α)×m_new
    
until convergence

Extract: s* = argmax belief(s)
```

**Pros**: Parallelizable | **Cons**: May not converge on loopy graphs

---

## Part 2: Controlled Test - Scrambled Signs

To fairly compare methods, we create a perfect director field and randomly scramble 50% of the signs. This creates a controlled test where we know the optimal solution.

In [2]:
# Create ground truth cholesteric director
SHAPE = (32, 32, 16)
PITCH = 6.0

print("Creating ground truth director...")
director_gt = fcpm.create_cholesteric_director(shape=SHAPE, pitch=PITCH)
n_gt = director_gt.to_array()
gt_energy = compute_gradient_energy(n_gt)

# Scramble signs randomly (50% flip)
print("Scrambling 50% of signs randomly...")
np.random.seed(42)
flip_mask = np.random.random(n_gt.shape[:3]) > 0.5
n_scrambled = n_gt.copy()
n_scrambled[flip_mask] = -n_scrambled[flip_mask]
director_scrambled = DirectorField.from_array(n_scrambled)
scrambled_energy = compute_gradient_energy(n_scrambled)

print(f"\n{'='*50}")
print(f"Ground Truth Energy:  {gt_energy:,.0f}")
print(f"Scrambled Energy:     {scrambled_energy:,.0f}")
print(f"Energy Gap to Close:  {scrambled_energy - gt_energy:,.0f}")
print(f"{'='*50}")

Creating ground truth director...
Scrambling 50% of signs randomly...

Ground Truth Energy:  19,456
Scrambled Energy:     98,334
Energy Gap to Close:  78,878


In [3]:
# Run all methods on scrambled data
scrambled_results = {}

methods = [
    ('V2 Layer+Refine', lambda d: layer_then_refine(d, verbose=False)),
    ('Graph Cuts', lambda d: GraphCutsOptimizer().optimize(d, verbose=False)),
    ('Hierarchical', lambda d: HierarchicalOptimizer().optimize(d, verbose=False)),
    ('Simulated Annealing', lambda d: SimulatedAnnealingOptimizer(
        SimulatedAnnealingConfig(max_iterations=50000, use_cluster_moves=True)
    ).optimize(d, verbose=False)),
    ('Belief Propagation', lambda d: BeliefPropagationOptimizer(
        BeliefPropagationConfig(max_iterations=100)
    ).optimize(d, verbose=False)),
]

print(f"{'Method':<25} {'Final Energy':>14} {'Recovery':>12} {'Time':>10}")
print("-"*65)

for name, method in methods:
    t0 = time.time()
    result = method(director_scrambled)
    elapsed = time.time() - t0
    
    recovery = 100 * (scrambled_energy - result.final_energy) / (scrambled_energy - gt_energy)
    
    status = " ★ OPTIMAL" if abs(result.final_energy - gt_energy) < 1 else ""
    
    scrambled_results[name] = {
        'energy': result.final_energy,
        'recovery': recovery,
        'time': elapsed,
        'director': result.director
    }
    
    print(f"{name:<25} {result.final_energy:>14,.0f} {recovery:>11.1f}% {elapsed:>9.2f}s{status}")

print("-"*65)
print(f"{'Ground Truth':<25} {gt_energy:>14,.0f} {'100.0':>11}% {'N/A':>10}")

Method                      Final Energy     Recovery       Time
-----------------------------------------------------------------
V2 Layer+Refine                   61,508        46.7%      0.05s
Graph Cuts                        19,456       100.0%      0.04s ★ OPTIMAL
Hierarchical                      29,696        87.0%      0.04s
Simulated Annealing               49,806        61.5%     24.77s
Belief Propagation                98,334         0.0%      0.01s
-----------------------------------------------------------------
Ground Truth                      19,456       100.0%        N/A


In [4]:
# Visualization: Energy Recovery Bar Chart
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

names = list(scrambled_results.keys())
recoveries = [scrambled_results[n]['recovery'] for n in names]
times = [scrambled_results[n]['time'] for n in names]

# Color by performance
colors = ['#2ecc71' if r > 95 else '#f39c12' if r > 50 else '#e74c3c' for r in recoveries]

# Recovery chart
ax = axes[0]
bars = ax.bar(range(len(names)), recoveries, color=colors, edgecolor='black', linewidth=1.2)
ax.axhline(y=100, color='green', linestyle='--', linewidth=2, label='Optimal (100%)')
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=30, ha='right')
ax.set_ylabel('Energy Recovery (%)')
ax.set_title('Energy Recovery from Scrambled Signs', fontsize=14, fontweight='bold')
ax.set_ylim(0, 110)
ax.legend(loc='lower right')

# Add value labels
for bar, val in zip(bars, recoveries):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
            f'{val:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Time chart
ax = axes[1]
bars = ax.bar(range(len(names)), times, color='steelblue', edgecolor='black', linewidth=1.2)
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=30, ha='right')
ax.set_ylabel('Time (seconds)')
ax.set_title('Computation Time', fontsize=14, fontweight='bold')
ax.set_yscale('log')

# Add value labels
for bar, val in zip(bars, times):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.2, 
            f'{val:.2f}s', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('scrambled_test_results.png', dpi=150, bbox_inches='tight')
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_21246/584005330.py:43: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


---

## Part 3: Realistic Test - Noisy Reconstruction

Now test with realistic FCPM simulation including noise.

In [5]:
# Realistic reconstruction with noise
NOISE_LEVEL = 0.05  # 5% noise

print("Simulating FCPM measurement...")
I_fcpm = fcpm.simulate_fcpm(director_gt)

print(f"Adding {NOISE_LEVEL*100:.0f}% noise...")
I_noisy = fcpm.add_fcpm_realistic_noise(I_fcpm, noise_model='mixed', gaussian_sigma=NOISE_LEVEL)
I_noisy = fcpm.normalize_fcpm(I_noisy)

print("Reconstructing via Q-tensor...")
director_raw, Q, info = reconstruct_via_qtensor(I_noisy)

raw_energy = compute_gradient_energy(director_raw.to_array())
raw_metrics = fcpm.summary_metrics(director_raw, director_gt)

print(f"\nRaw reconstruction (no sign fix):")
print(f"  Energy: {raw_energy:,.0f}")
print(f"  Angular error: {raw_metrics['angular_error_mean_deg']:.2f}°")

Simulating FCPM measurement...
Adding 5% noise...
Reconstructing via Q-tensor...

Raw reconstruction (no sign fix):
  Energy: 36,872
  Angular error: 7.24°


In [6]:
# Test methods on noisy reconstruction
noisy_results = {}

print(f"{'Method':<25} {'Energy':>12} {'Ang.Error':>12} {'Time':>10}")
print("-"*60)

for name, method in methods:
    t0 = time.time()
    result = method(director_raw)
    elapsed = time.time() - t0
    
    metrics = fcpm.summary_metrics(result.director, director_gt)
    
    noisy_results[name] = {
        'energy': result.final_energy,
        'error': metrics['angular_error_mean_deg'],
        'time': elapsed,
        'director': result.director
    }
    
    print(f"{name:<25} {result.final_energy:>12,.0f} {metrics['angular_error_mean_deg']:>11.2f}° {elapsed:>9.2f}s")

print("-"*60)
print(f"{'Raw (no fix)':<25} {raw_energy:>12,.0f} {raw_metrics['angular_error_mean_deg']:>11.2f}° {'N/A':>10}")

Method                          Energy    Ang.Error       Time
------------------------------------------------------------
V2 Layer+Refine                 20,877        7.24°      0.00s
Graph Cuts                      20,877        7.24°      0.04s
Hierarchical                    31,221        7.24°      0.05s
Simulated Annealing             21,129        7.24°     11.53s
Belief Propagation              36,872        7.24°      0.01s
------------------------------------------------------------
Raw (no fix)                    36,872        7.24°        N/A


---

## Part 4: Visual Comparison

In [7]:
# Director slice visualization
z_mid = SHAPE[2] // 2

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Top row
fcpm.plot_director_slice(director_gt, z_idx=z_mid, step=2, ax=axes[0, 0], 
                         title='Ground Truth')
fcpm.plot_director_slice(director_scrambled, z_idx=z_mid, step=2, ax=axes[0, 1], 
                         title='Scrambled Input')
fcpm.plot_director_slice(scrambled_results['Graph Cuts']['director'], z_idx=z_mid, step=2, 
                         ax=axes[0, 2], title='Graph Cuts (100% recovery)')

# Bottom row
fcpm.plot_director_slice(scrambled_results['V2 Layer+Refine']['director'], z_idx=z_mid, step=2, 
                         ax=axes[1, 0], title=f"V2 Layer+Refine ({scrambled_results['V2 Layer+Refine']['recovery']:.0f}%)")
fcpm.plot_director_slice(scrambled_results['Hierarchical']['director'], z_idx=z_mid, step=2, 
                         ax=axes[1, 1], title=f"Hierarchical ({scrambled_results['Hierarchical']['recovery']:.0f}%)")
fcpm.plot_director_slice(scrambled_results['Simulated Annealing']['director'], z_idx=z_mid, step=2, 
                         ax=axes[1, 2], title=f"Simulated Annealing ({scrambled_results['Simulated Annealing']['recovery']:.0f}%)")

plt.suptitle(f'Director Field Comparison at z={z_mid}', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('director_slices_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_21246/3889862284.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


---

## Part 5: Noise Sensitivity Analysis

In [8]:
# Quick noise sensitivity test
noise_levels = [0.01, 0.03, 0.05, 0.08, 0.10]

# Only test fast methods for presentation
fast_methods = [
    ('V2 Layer+Refine', lambda d: layer_then_refine(d, verbose=False)),
    ('Graph Cuts', lambda d: GraphCutsOptimizer().optimize(d, verbose=False)),
    ('Hierarchical', lambda d: HierarchicalOptimizer().optimize(d, verbose=False)),
]

noise_data = {name: {'errors': [], 'energies': []} for name, _ in fast_methods}

print("Running noise sensitivity analysis...\n")

for noise in noise_levels:
    print(f"Noise {noise*100:.0f}%: ", end="")
    
    I_noisy = fcpm.add_fcpm_realistic_noise(I_fcpm, noise_model='mixed', gaussian_sigma=noise)
    I_noisy = fcpm.normalize_fcpm(I_noisy)
    director_raw_n, _, _ = reconstruct_via_qtensor(I_noisy)
    
    for name, method in fast_methods:
        result = method(director_raw_n)
        metrics = fcpm.summary_metrics(result.director, director_gt)
        noise_data[name]['errors'].append(metrics['angular_error_mean_deg'])
        noise_data[name]['energies'].append(result.final_energy)
    
    print(f"done")

print("\nComplete!")

Running noise sensitivity analysis...

Noise 1%: done
Noise 3%: done
Noise 5%: done
Noise 8%: done
Noise 10%: done

Complete!


In [9]:
# Plot noise sensitivity
fig, ax = plt.subplots(figsize=(10, 6))

colors = {'V2 Layer+Refine': '#3498db', 'Graph Cuts': '#2ecc71', 'Hierarchical': '#9b59b6'}
markers = {'V2 Layer+Refine': 'o', 'Graph Cuts': 's', 'Hierarchical': '^'}

noise_pct = [n * 100 for n in noise_levels]

for name in noise_data:
    ax.plot(noise_pct, noise_data[name]['errors'], 
            marker=markers[name], color=colors[name],
            linewidth=2.5, markersize=10, label=name)

ax.set_xlabel('Noise Level (%)', fontsize=12)
ax.set_ylabel('Mean Angular Error (°)', fontsize=12)
ax.set_title('Angular Error vs Noise Level', fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 11)

plt.tight_layout()
plt.savefig('noise_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_21246/1433107349.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


---

## Part 6: Summary & Conclusions

### Performance Ranking

| Rank | Method | Energy Recovery | Speed | Recommendation |
|------|--------|-----------------|-------|----------------|
| **1** | **Graph Cuts** | **100%** (Optimal) | Fast (0.04s) | **Best choice** |
| 2 | Hierarchical | 87% | Fast (0.04s) | Good alternative |
| 3 | V2 Layer+Refine | 47% | Very Fast (0.03s) | Quick baseline |
| 4 | Simulated Annealing | 30-62% | Very Slow (28s) | Not recommended |
| 5 | Belief Propagation | 0% | Fast (0.01s) | Needs tuning |

### Key Insights

1. **Graph Cuts achieves globally optimal solution** - Uses min-cut/max-flow to solve exactly

2. **Hierarchical provides 87% recovery** - Q-tensor averaging at coarse scales captures global structure

3. **Simulated Annealing is too slow** - Even with 50k iterations, doesn't match deterministic methods

4. **All methods achieve similar angular error on noisy data** - Sign optimization primarily affects energy, not direction accuracy

### Recommended Workflow

```python
from v2.approaches import GraphCutsOptimizer

# Default: Use Graph Cuts
optimizer = GraphCutsOptimizer()
result = optimizer.optimize(director_raw, verbose=True)
optimized = result.director
```

In [10]:
# Final summary table
print("\n" + "="*70)
print("FINAL SUMMARY: SCRAMBLED SIGNS TEST")
print("="*70)
print(f"\n{'Method':<25} {'Recovery':>12} {'Time':>12} {'Verdict':>15}")
print("-"*70)

verdicts = {
    'Graph Cuts': 'BEST',
    'Hierarchical': 'Good',
    'V2 Layer+Refine': 'Baseline',
    'Simulated Annealing': 'Slow',
    'Belief Propagation': 'Failed'
}

for name in scrambled_results:
    r = scrambled_results[name]
    print(f"{name:<25} {r['recovery']:>11.1f}% {r['time']:>11.2f}s {verdicts[name]:>15}")

print("="*70)
print("\nRecommendation: Use Graph Cuts (PyMaxflow) for optimal results.")


FINAL SUMMARY: SCRAMBLED SIGNS TEST

Method                        Recovery         Time         Verdict
----------------------------------------------------------------------
V2 Layer+Refine                  46.7%        0.05s        Baseline
Graph Cuts                      100.0%        0.04s            BEST
Hierarchical                     87.0%        0.04s            Good
Simulated Annealing              61.5%       24.77s            Slow
Belief Propagation                0.0%        0.01s          Failed

Recommendation: Use Graph Cuts (PyMaxflow) for optimal results.


In [11]:
print("\nPresentation notebook complete!")
print("\nGenerated figures:")
print("  • scrambled_test_results.png")
print("  • director_slices_comparison.png")
print("  • noise_sensitivity.png")


Presentation notebook complete!

Generated figures:
  • scrambled_test_results.png
  • director_slices_comparison.png
  • noise_sensitivity.png
